Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\SSAFY/.cache\torch\hub\master.zip
YOLOv5  2023-4-26 Python-3.11.2 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2070 Super with Max-Q Design, 8192MiB)



requirements: C:\Users\SSAFY\.cache\torch\hub\requirements.txt not found, check failed.


Fusing layers... 
one_YOLOv5m summary: 212 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


## 모델로 바운딩 박스 그리기

Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 im

## 수작업으로 모델 조차 이상하게 그린 바운딩 박스 이미지 제거

### 0과 1을 대조하여 둘 다 존재하는 이미지의 원본 2로 복사

#### 그러나 이전 모델 학습에 사용한 데이터가 있으면 지워야지!

In [16]:
pre_img = os.listdir('./2_learning_img/')

for i in pre_img:
    os.remove(f'./2_learning_img/{i}')

In [3]:
bounding_processing_img = os.listdir('./1_bounding_img')

scrapping_img = os.listdir('./0_scrapping_img')

for spi in scrapping_img:
    if spi in bounding_processing_img:
        shutil.copy(f'./0_scrapping_img/{spi}', './2_learning_img')

## 데이터 증강

### 증강 함수 정의

In [4]:
def adjust_hue(image):
    hue_shift = random.randint(-20, 20)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_image[:,:,0] = (hsv_image[:,:,0] + hue_shift) % 180
    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

# image = cv2.imread('image.jpg')
# hue_adjusted_image = adjust_hue(image)
# cv2.imwrite('hue_adjusted_image.jpg', hue_adjusted_image)

## 데이터 세트화 및 라벨 생성

#### 이전 3의 데이터 삭제

In [15]:
for C in ['train', 'test']:
    for L in ['labels', 'images']:
        for i in os.listdir(f'./3_one_time_data/{C}/{L}'):
            os.remove(f'./3_one_time_data/{C}/{L}/{i}')
    
    if os.path.isfile(f'./3_one_time_data/{C}/labels.cache'):
        os.remove(f'./3_one_time_data/{C}/labels.cache')

In [5]:
image_list = os.listdir(f'./2_learning_img/')

test_set = {}
for i in random.sample(image_list, len(image_list)//10 *2):
    test_set[i] = True

for f_name in image_list:
    
    if test_set.get(f_name):
        CLS = 'test'
    else:
        CLS = 'train'
    
    img = Image.open(f'./2_learning_img/{f_name}')
    # cv_img = cv2.imread(f'./2_learning_img/{f_name}')
    
    results = model(img)
    
    if len(results.xywh[0]):
        
        for result in results.xywhn[0]:
            x,y,w,h,score,cls = map(float, result)
        
            with open(f'./3_one_time_data/{CLS}/labels/{f_name[:-4]}.txt', 'a') as txt:
                txt.write(f'{cls} {round(x, 4)} {round(y, 4)} {round(w, 4)} {round(h, 4)}\n')
        else:
            shutil.copy(f'./2_learning_img/{f_name}', f'./3_one_time_data/{CLS}/images/{f_name}')
    
    else:
        continue

    
    # adjust_col_img = adjust_hue(cv_img)
    # cv2.imwrite(f'./3_one_time_data/{CLS}/images/{f_name[:-4]}_c.jpg', adjust_col_img)

    # shutil.copy(f'./3_one_time_data/{CLS}/labels/{f_name[:-4]}.txt', f'./3_one_time_data/{CLS}/labels/{f_name[:-4]}_c.txt')

In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data one_data.yaml --cfg one_yolov5m.yaml --weight yolov5m.pt --name rose_model

In [ ]:
## 이상 없으면 3 -> 5 보내버리기!
for CLS in ['train', 'test']:
    
    for i in ['images', 'labels']:
        
        for f in os.listdir(f'./3_one_time_data/{CLS}/{i}'):
            
            shutil.move(f'./3_one_time_data/{CLS}/{i}/{f}', f'./5_total_data/{CLS}/{i}/{f}')

In [14]:
img = cv2.imread('./0_scrapping_img/rose_0040.jpg')

img.shape

(179, 282, 3)